In [1]:
import numpy as np
import os 
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy import ndimage

def plot_dem_and_mask(dem_data, null_mask):
    # Define a custom colormap for the mask: blue for valid data, red for no-data
    mask_cmap = ListedColormap(['blue', 'red'])
    
    # Calculate vmin and vmax based on the 5th and 85th percentiles (ignoring NaNs)
    flat_data = dem_data[~np.isnan(dem_data)]
    vmin = np.percentile(flat_data, 5)
    vmax = np.percentile(flat_data, 85)
    
    # Create a figure with two subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot DEM
    im1 = axes[0].imshow(
        dem_data,
        cmap='terrain',
        vmin=vmin,
        vmax=vmax
    )
    # Set values below zero to black
    im1.cmap.set_under('black')
    
    axes[0].set_title("DEM (Terrain)")
    fig.colorbar(im1, ax=axes[0], orientation='vertical', extend='both')
    
    # Plot Null Mask
    im2 = axes[1].imshow(
        null_mask,
        cmap=mask_cmap,
        vmin=0,
        vmax=1
    )
    axes[1].set_title("Null Mask (Blue=Data, Red=No Data)")
    
    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()

# Load raster
def rload(dem_path):
    with rasterio.open(dem_path) as src:
        rdata = src.read()
    return rdata

# Read DEM and replace nodata values with NaN
def read_dem(dem_path):
    with rasterio.open(dem_path) as src:
        dem_data = src.read(1)
        nodata_value = src.nodata
    if nodata_value is not None:
        dem_data = dem_data.astype("float32")
        dem_data[dem_data == nodata_value] = np.nan
        dem_data[dem_data <= -30] = np.nan
    return dem_data

# Get mask for NaN values
def get_null_mask(dem_data):
    return np.isnan(dem_data)

# Print unique values
# def print_unique_values(array, verbose=False):
#     unique_values, counts = np.unique(array, return_counts=True)
#     print(f"Total Unique Values: {len(unique_values)}")
#     print(unique_values)
#     if verbose:
#         print("Unique Values and Counts:")
#         for value, count in zip(unique_values, counts):
#             print(f"Value: {value}, Count: {count}")


def print_unique_values(array, verbose=False):
    unique_values, counts = np.unique(array, return_counts=True)
    total_count = np.sum(counts)

    print(f"Total Unique Values: {len(unique_values)}")
    print(unique_values)

    if verbose:
        print("Unique Values and Percentages:")
        for value, count in zip(unique_values, counts):
            percentage = (count / total_count) * 100
            print(f"Value: {value}, Percentage: {percentage:.2f}%")

# Save the masks and filtered DEM
def save_raster(output_path, data, reference_raster):
    """Save raster using reference metadata."""
    with rasterio.open(reference_raster) as src:
        meta = src.meta.copy()
    meta.update(dtype="float32", nodata=np.nan)
    with rasterio.open(output_path, "w", **meta) as dst:
        dst.write(data.astype("float32"), 1)



In [2]:
hem_fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_hem.tif"
wam_fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_wam.tif"
com_fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_com.tif"
lcm_fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_edem_lcm.tif"
wbm_fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_cdem_wbm.tif"
esa_fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_esawc.tif"
tdem_fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N10E105/N10E105_tdem_dem_egm.tif"

In [ ]:
from

In [ ]:
from..src

In [5]:
from ..src.rfilter import dem_remove_badpixels
fo = tdem_fn.replace(".tif", "_void.tif")

ImportError: attempted relative import with no known parent package

In [ ]:
dem_remove_badpixels(tdem_fn,hem_fn, esa_fn,com_fn,fo)

In [35]:
dem_data = read_dem(tdem_fn)
hem = rload(hem_fn) #
wam = rload(wam_fn) #
com = rload(com_fn) #
lcm = rload(lcm_fn)
esa = rload(esa_fn)
wbm = rload(wam_fn)

In [ ]:

def filter_tandemx_pixelnoise(tdem_fn,hem_fn,com_fn,esa_fn, n_iter=2):
    dem_data = read_dem(tdem_fn)
    hem = rload(hem_fn) #
    esa = rload(esa_fn)
    com = rload(com_fn) 

    print("# Apply height error mask")
    mask = get_null_mask(dem_data) #
    max_err_multi = 0.5#1#1.5#2#1.5#1.5
    n_iter = 1
    print_unique_values(mask, verbose=True)
    save_raster("mask.tif", mask.astype("uint8"), tdem_fn)
    print("# Apply height error mask")
    # Apply height error mask
    mask |= (hem[0] > max_err_multi)
    print_unique_values(mask, verbose=True)

    #print("# Apply COM mask (invalid values 0, 1, 2)")
    com_invalid = (0, 1, 2)
    mask |= np.isin(com[0], com_invalid)
    print_unique_values(mask, verbose=True)

    #print("# Apply ESA water mask (select only 80 for water)")
    mask |= (esa[0] == 80)
    print_unique_values(mask, verbose=True)
    save_raster("mask1.tif", mask.astype("uint8"), tdem_fn)
    print("# Apply Morphological Operations")
    # ==========================================
    mask = ndimage.binary_dilation(mask, iterations=n_iter)
    mask = ndimage.binary_erosion(mask, iterations=n_iter)

In [77]:
print("# Apply height error mask")
mask = get_null_mask(dem_data) #
max_err_multi = 0.5#1#1.5#2#1.5#1.5
n_iter = 1
print_unique_values(mask, verbose=True)
save_raster("mask.tif", mask.astype("uint8"), tdem_fn)
print("# Apply height error mask")
# Apply height error mask
mask |= (hem[0] > max_err_multi)
print_unique_values(mask, verbose=True)

#print("# Apply COM mask (invalid values 0, 1, 2)")
com_invalid = (0, 1, 2)
mask |= np.isin(com[0], com_invalid)
print_unique_values(mask, verbose=True)

#print("# Apply ESA water mask (select only 80 for water)")
mask |= (esa[0] == 80)
print_unique_values(mask, verbose=True)
save_raster("mask1.tif", mask.astype("uint8"), tdem_fn)

print("# Apply Morphological Operations")
# ==========================================
mask = ndimage.binary_dilation(mask, iterations=n_iter)
mask = ndimage.binary_erosion(mask, iterations=n_iter)

save_raster("mask3.tif", mask.astype("uint8"), tdem_fn)

# mask has True and False, and i want to correct the code below
# Apply Mask to DEM: Replace masked values with np.nan or -9999
filtered_dem = np.where(mask, np.nan, dem_data)  # Use np.nan for float-type DEM
# filtered_dem = np.where(mask, -9999, dem_data)  # Use -9999 for integer-type DEM

# Save the filtered DEM to a new raster file
save_raster("filtered_dem.tif", filtered_dem, tdem_fn)

# Apply height error mask
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 98.95%
Value: True, Percentage: 1.05%
# Apply height error mask
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 75.89%
Value: True, Percentage: 24.11%
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 75.83%
Value: True, Percentage: 24.17%
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 75.40%
Value: True, Percentage: 24.60%
# Apply Morphological Operations


# Apply Mask to DEM


In [75]:
mask

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False,  True,  True, ...,  True,  True, False],
       [False,  True,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
print("# Apply height error mask")
mask = get_null_mask(dem_data) #
max_err_multi = 0.5#1#1.5#2#1.5#1.5
n_iter = 1
print_unique_values(mask, verbose=True)
save_raster("mask.tif", mask.astype("uint8"), tdem_fn)
print("# Apply height error mask")
# Apply height error mask
mask |= (hem[0] > max_err_multi)
print_unique_values(mask, verbose=True)

#print("# Apply COM mask (invalid values 0, 1, 2)")
com_invalid = (0, 1, 2)
mask |= np.isin(com[0], com_invalid)
print_unique_values(mask, verbose=True)

#print("# Apply ESA water mask (select only 80 for water)")
mask |= (esa[0] == 80)
print_unique_values(mask, verbose=True)
save_raster("mask1.tif", mask.astype("uint8"), tdem_fn)

print("# Apply Morphological Operations")
# ==========================================
mask = ndimage.binary_dilation(mask, iterations=n_iter)
mask = ndimage.binary_erosion(mask, iterations=n_iter)

save_raster("mask3.tif", mask.astype("uint8"), tdem_fn)

# hem is the super thingy, and use copernicus as 

# mask = (lcm[0] == 3)   # Land class in LCM Band 1
# print_unique_values(mask, verbose=True)
# save_raster("mask2.tif", mask.astype("uint8"), tdem_fn)

# print("# Apply WBM water mask (select 1, 2, 3 for water")
# #mask |= np.isin(wbm[0], [1, 2, 3])
# mask |= (wbm[0] == 1)
# print_unique_values(mask, verbose=True)
# save_raster("mask.tif", mask.astype("uint8"), tdem_fn)
# # this is wrong one

# print("# Apply WAM mask (range 3 to 127)//33 to 127")
# wam_clim = (33, 127)
# mask |= (wam[0] >= wam_clim[0]) & (wam[0] <= wam_clim[1])
# print_unique_values(mask, verbose=True)
# save_raster("mask2.tif", mask.astype("uint8"), tdem_fn)

# mask 1 is good, mask 2 is too much - removing even good pixels 

# Apply height error mask
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 98.95%
Value: True, Percentage: 1.05%
# Apply height error mask
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 75.89%
Value: True, Percentage: 24.11%
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 75.83%
Value: True, Percentage: 24.17%
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 75.40%
Value: True, Percentage: 24.60%
Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 97.13%
Value: True, Percentage: 2.87%


Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 92.09%
Value: True, Percentage: 7.91%


Total Unique Values: 2
[False  True]
Unique Values and Percentages:
Value: False, Percentage: 59.97%
Value: True, Percentage: 40.03%
